# Imports

In [6]:
#! pip install gmaps
#! pip install google

In [ ]:
#! jupyter nbextension enable --py gmaps

In [3]:
with open('./map_api_key.rtf') as f:
    api_key = f.readline()
    f.close

In [8]:
import gmaps
import google
import requests
import gmaps.datasets

import pandas as pd
import numpy as np

gmaps.configure(api_key='<ENTER YOUR KEY HERE>')

In [ ]:
# Get dataframes from csv
df_roads = pd.read_csv('./datasets/roads.csv')
df = pd.read_csv('./datasets/location_extracted.csv')
df_context = pd.read_csv('./datasets/location_context.csv')

# Mapping

We're using the Google Places API to get latitude and longitude coordinates to map exact road locations. 
The goal here is to highlight precise roads where assistance would be needed so that first respondants could be able to location people in need of assistance

In [4]:
# SOURCE https://www.shanelynn.ie/batch-geocoding-in-python-with-google-geocoding-api/
# function that takes in as arguments a address and returns the latitude and longitude

def get_google_latlong(address, api_key=None, return_full_response=False):
    # Set up your Geocoding url
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    # Ping google for the reuslts:
    results = requests.get(geocode_url)
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "latitude": None,
            "longitude": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng')
        }
        
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

In [9]:
# ENHANCED MODEL MAP
# Loop through the dataframe with the locations extracted and place a pin on the map for every one of them
# this gets the coordinated of each address from the api using the function above


# we want to center the map on Butte county so we need the coordinates to set the center
butte_coordinates = (39.722496, -122.1348896)
fig = gmaps.figure(center=butte_coordinates, 
             zoom_level=9, 
             map_type='TERRAIN', 
             layout={'width': '650px', 'height': '550px'})


# Loop through the results
for i in range(df.shape[0]):
    list_loc = df_context['location_extraction'].iloc[i].split(' , ')
    locations = []
    for i in range(len(list_loc)):
        x, y = (get_google_latlong(list_loc[i], api_key='<ENTER YOUR KEY HERE>')['latitude'], 
                get_google_latlong(list_loc[i], api_key='<ENTER YOUR KEY HERE>')['longitude'])
        if (x != None) & (y != None):
            locations.append((x, y))
    if len(locations) != 0:
        heatmap = gmaps.heatmap_layer(locations=locations)
        fig.add_layer(heatmap)
    
fig

0
100
200
300
400
500
600
700
800
900


Figure(layout=FigureLayout(height='550px', width='650px'))